In [1]:
%pylab inline
import sys

Populating the interactive namespace from numpy and matplotlib


# The total chromaticity is given by :

# $ \xi_{tot} = \frac{1}{4\pi} \oint [ m (s)D(s)+k(s)] \beta(s) ds $

## Where m is the strength of the sextupole; D is the dispersion function; k is the quadrupole strength; $\beta$ is the beta function 

## There exists a linear relationship between the chromaticity and the sextupole strength, and the slope is proportional to the product of D and $\beta$

$  $

## $\begin{cases} \begin{align*} \xi_x &= \sum_im_i x_i + x_0 \\
                              \xi_y &= \sum_i n_i x_i + y_0 \end{align*}\end{cases}$ 


##  Where $\xi_x$, $\xi_y$ is the first order chromaticity

## $x_i$ is the strength of sextupole

## $m_i$, $ n_i$ only depends on local optical functions



$ $

In [2]:
m = array([0.6909, 6.3624, 0.6912, 0.6915, 6.3631, 0.6916])       #coefficient of the first equation
n = array([-1.7945, -1.9225, -1.7601, -1.7911, -1.9199, -1.7599]) # coefficient of the second equation

CX = 3.953515      #orginal chrom.x
CY = 6.595477      #orginal chrom.y

In [3]:
def cx(x):         #compute chrom.x
    return dot(m,x) + x0     

def cy(x):         #compute chrom.y
    return dot(n,x) + y0    

$  $

# let $\xi'_x = \xi_x - x_0 $, and  $\xi'_y = \xi_y - y_0 $

## $\begin{cases} \begin{align*} \xi'_x &= m_1 x_1+m_2 x_2 + ...+ m_6 x_6  \\
                              \xi'_y &= n_1 x_1+n_2 x_2 + ...+ n_6 x_6  \end{align*}\end{cases}$ 

# 6 variables to be determined and exists 2 equations, so there are 4 free variables
                              
## Manipulate the second equation, find $x_1$ as a function of other variables

## $x_1 = \frac{\xi'_y - n_2 x_2 - ...- n_6 x_6}{n_1}$, subsitute it into the first equation 

## $\xi'_x - \frac{m_1 \xi'_y}{n_1} = (m_2-\frac{m_1 n_2}{n_1}) x_2 +(m_3-\frac{m_1 n_3}{n_1}) x_3 + ...+ (m_6-\frac{m_1 n_6}{n_1}) x_6$

In [4]:
coeff = array([m[i]-m[0]*n[i]/n[0] for i in arange(1,6)])   # coefficients of equation above

print('the coefficients are :' ,coeff)

the coefficients are : [5.62221875e+00 1.35443355e-02 1.90903316e-03 5.62391978e+00
 1.40213374e-02]


In [5]:
def func(X0,Y0,x): # function to comptue x1 and x2; x is the array contains 4 free variables (x3,x4,x5,x6)
    Xp = CX - X0   #Chrom'.x
    Yp = CY - Y0   #Chrom'.y
    lhs = Xp - m[0]*Yp/n[0]                 # compute LHS in the equation above
    x2 = (lhs-dot(coeff[1:],x))/coeff[0]    # variable x2
    x1 = (Yp - n[1]*x2- dot(n[2:],x))/n[0]  # variable x1
    return x1, x2

# To solve for the equation at minimum strength:

## Step 1: First generate a data array for 4 free variables within a resonable range

## Step 2: Pick a number from each data array to generate a data group for those 4 variables $\{x_3, x_4, x_5, x_6\}$

## Step3: Compute $x_1, x_2$ and only record solutions in the required range 

## Step4: Find the maximum strength in each solution group {$x_1,x_2, x_3, x_4, x_5, x_6$}

## Step5: Make a comparison between these max to find the minimum group, which is the expected solution

## Step6: Do iteration, use the solution to gradually narrow dataset range 
## e.g. let's say, in first iteration, we find $x_3$ = 3.1. The initial data range of $x_3$ is $[-5.5,5.5]$, now we can add $\pm 0.5$ to 3.1.  Narrow this range to $[2.6,3.6]$, and  use it for the next iteration.

## Optimizer   

In [6]:
def multsolver(X0,Y0,a,b,c,d):

    xxi = []
    
    for i in range(num):
        for j in range(num): 
            for k in range(num):
                for l in range(num):
                    
                    xx = array([a[i],b[j],c[k],d[l]]) # record the 4 variables
                    x1 = func(X0,Y0,xx)[0]      # compute x1
                    x2 = func(X0,Y0,xx)[1]      # compute x2
                    
                    if abs(x1) <=4.5 and abs(x2) <=4.5: # pick up the solutions in required range
                        xs = concatenate((x1,x2),axis=None)
                        xxi.append(concatenate((xs,xx),axis=None))
                        
    xxi = array(xxi)
    
    xxim = []
    for i in range(len(xxi)):
        xxim.append(max(abs(xxi[i]))) # find the maxium value of 6 ariables in each solution
    xxim = array(xxim)
    
    if len(xxim)==0:
        sys.exit('please increase the maximum strength')

    nm = where(xxim == min(xxim))[0][0]      # find where the minimum solution is
    return xxi[nm]

In [7]:
def opt(x0,y0,mm,ni,num):   # sextupole strength optimizer
    
    ama =  mm   ## max of the data 
    ami = -mm   ## min of the data 
    bma =  mm
    bmi = -mm
    cma =  mm
    cmi = -mm
    dma =  mm
    dmi = -mm
    
    sa = []  # initialize solution array
    msa = [] # initialize maximum strength array
    
    for i in range(ni):
        a = linspace(ami,ama,num)      # generating data within the required range 
        b = linspace(bmi,bma,num)
        c = linspace(cmi,cma,num)
        d = linspace(dmi,dma,num)
        
        s = multsolver(x0,y0,a,b,c,d) # compute x1,x2
    
        ms = max(abs(s))              # find the max in solution array{x1,x2,x3,x4,x5,x6}
    
        ra = 1- i/5                   # narrowing the variation range of data array
    
        if ra<0.02:                   # set minimum variation range
            ra=0.02

        if i!= 0 and ms > msa[-1]:    # if iteration step finds a bigger strength, 
                                      #go back to previous solution and increase num
            num += 1
        
            ama = sa[-1][2]+ra
            ami = sa[-1][2]-ra
        
            bma = sa[-1][3]+ra
            bmi = sa[-1][3]-ra
        
            cma = sa[-1][4]+ra
            cmi = sa[-1][4]-ra
        
            dma = sa[-1][5]+ra
            dmi = sa[-1][5]-ra
            
        else:
            ama = s[2]+ra
            ami = s[2]-ra
        
            bma = s[3]+ra
            bmi = s[3]-ra
        
            cma = s[4]+ra
            cmi = s[4]-ra
        
            dma = s[5]+ra
            dmi = s[5]-ra
        
        if num > 16:    #insurance of efficiency
            num = 16
            
        sa.append(s)
        msa.append(ms)

        print('iteration step: %s'%(i+1))
        print('the max strength is:',ms)
        print(s)
        print(num)
        print('')
        print('')
    
    sa = array(sa)
    msa = array(msa)

    nm = where(msa ==min(msa))[0][0]      # find the where the minimum solution is

    xs = sa[nm]
    print('the iretation step reached mininum:',nm+1)
    print('the max strength in the solution is:',min(msa))
    return xs

## Equation for tuning Slx and Sly

## $\begin{cases} \begin{align*} x_0 &= 0.9253 \textbf{Slx} -0.0109 \textbf{Sly} - 8.677114 \\
                                  y_0 &= -0.0123 \textbf{Slx} +10.038 \textbf{Sly}- 32.160215 \end{align*}\end{cases}$

## Where Slx is the percentage change on SLX* sextupoles and Sly for SLY* sextupoles

In [8]:
def xg(sx,sy):
    return 0.9253*sx-0.0109*sy - 8.677114     

def yg(sx,sy):
    return -0.0123*sx+10.038*sy - 32.160215  

In [9]:
Sx = 0 # percentage increment on Slx
Sy = 1.7 #percentage increment on Sly

In [10]:
x0 = xg(Sx,Sy) 
y0 = yg(Sx,Sy) 
print('x0:',x0)
print('y0:',y0)

x0: -8.695644
y0: -15.095615000000002


In [11]:
mm = 4     # max sextupole strength
ni = 12    # number of iterations to do
num  = 12  # number of data to be generated, total combination n^4

In [12]:
solution = opt(x0,y0,mm,ni,num)

iteration step: 1
the max strength is: 4.265987825044221
[-4.26598783 -0.24498146 -4.         -4.          4.         -4.        ]
12


iteration step: 2
the max strength is: 4.090909090909091
[-3.99921192  0.75579769 -4.09090909 -4.09090909  3.         -4.09090909]
12


iteration step: 3
the max strength is: 4.163636363636364
[-3.7857912   1.55642101 -4.16363636 -4.16363636  2.2        -4.16363636]
13


iteration step: 4
the max strength is: 4.090909090909091
[-4.00027573  1.35597923 -4.09090909 -4.09090909  2.4        -4.09090909]
13


iteration step: 5
the max strength is: 4.090909090909091
[-4.00098493  1.75610025 -4.09090909 -4.09090909  2.         -4.09090909]
13


iteration step: 6
the max strength is: 4.090909090909091
[-4.00133953  1.95616076 -4.09090909 -4.09090909  1.8        -4.09090909]
13


iteration step: 7
the max strength is: 4.074242424242424
[-4.05060896  1.97607943 -4.07424242 -4.07424242  1.78       -4.07424242]
13


iteration step: 8
the max strength is: 4.0702729

In [13]:
solution #solution array

array([-4.07015479,  1.89268594, -4.06757576, -4.06757576,  1.86333333,
       -4.06757576])

### Orginal chromaticity  $\xi_x = 3.953515$   

In [14]:
cx(solution) # testing the solution

3.9535149999999977

### Orginal chromaticity $ \xi_y = 6.595477 $

In [15]:
cy(solution)

6.595476999999999

## When this solution passes the test, we can use Tao to further optimize it. It won't take too long, since it's close to  a true solution.